Rum Yue
### Basics: Select, From, Where

In [75]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import os

In [77]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/xlyue/Documents/kaggle learning/Intro to SQL/sql-bq-244804-803e47cbe74e.json"
client = bigquery.Client()

In [97]:
dataset_ref = client.dataset('openaq', project = 'bigquery-public-data')
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [98]:
tables = list(client.list_tables(dataset))
len(tables)

1

In [43]:
for table in tables:
    print(table.table_id)

global_air_quality


In [99]:
table_ref = dataset_ref.table('global_air_quality')
table = client.get_table(table_ref)
client.list_rows(table, max_results = 5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [45]:
# pick data rows for US
query = """
           select city
           from `bigquery-public-data.openaq.global_air_quality`
           where country = 'US'
            """

In [46]:
query_job = client.query(query)
# return a pandas DataFrame
us_cities = query_job.to_dataframe()

In [58]:
us_cities.city.value_counts().reset_index().head()

,index,city
0,Phoenix-Mesa-Scottsdale,87
1,Houston,82
2,Los Angeles-Long Beach-Santa Ana,63
3,New York-Northern New Jersey-Long Island,60
4,Riverside-San Bernardino-Ontario,59


In [59]:
# easy implementation for multiple columns
query = """
           select city, country
           from `bigquery-public-data.openaq.global_air_quality`
           where country = 'US'
            """

In [60]:
# all columns
query = """
           select *
           from `bigquery-public-data.openaq.global_air_quality`
           where country = 'US'
            """

Note that SQL does not require capitalize operators.

In [93]:
# scan dataset size before running
query = """
           select score, title
           from `bigquery-public-data.hacker_news.full`
           where type = 'job'
            """

In [88]:
# estimate the size of a query
dry_run_config = bigquery.QueryJobConfig(dry_run = True)
dry_run_query_job = client.query(query, job_config = dry_run_config)

In [90]:
print('The query above will process {} bytes.'.format(dry_run_query_job.total_bytes_processed))

The query above will process 394726615 bytes.


In [96]:
# limit size to 1 MB
ohm = 1 * 1000 * 1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = ohm)
safe_query_job = client.query(query, job_config = safe_config)

In [84]:
# run the query within 1 GB size
ogb = 1000 * 1000 * 1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = ogb)

In [85]:
safe_query_job = client.query(query, job_config = safe_config)

In [86]:
job_post_scores = safe_query_job.to_dataframe()

In [87]:
job_post_scores.score.mean()

1.9628760215614676

Exercises

In [103]:
# Which countries have reported pollution levels in units of "ppm"?  
# In the code cell below, set `first_query` to an SQL query that pulls the appropriate entries from the `country` column.
first_query = """
                  select country 
                  from `bigquery-public-data.openaq.global_air_quality`
                  where unit = 'ppm'
                  """

In [104]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
first_results = first_query_job.to_dataframe()

# View top few rows of results
print(first_results.head())

  country
0      US
1      US
2      US
3      US
4      US


In [107]:
# Which pollution levels were reported to be exactly 0? 
zero_pollution_query = """
                                select * 
                                from `bigquery-public-data.openaq.global_air_quality`
                                where value = 0
                                """
query_job = client.query(zero_pollution_query, job_config=safe_config)